In [ ]:
# the source graph was prepared using the map taken from:
# https://github.com/pszufe/OpenStreetMapX.jl/blob/master/test/data/reno_east3.osm

## path to datasets
datadir='../Datasets/'

## package to add:
## conda install folium


In [ ]:
import folium as flm 
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Major limitations of the simplified simulation
* real road graphs are directed
* we ignore number of lanes
* we ignore speed limits
* we ignore restrictions on turning on intersections
* we ignore street lights
* we ignore road usage restrictions
* we ignore road class
* we assume uniform source and destination locations

We build two variants: ignoring and not ingoring road length

In [ ]:
## build undirected weighted graph
g_edges = pd.read_csv(datadir+'Reno/weights.csv')
nv = 1 + max(max(g_edges["from"]), max(g_edges["to"]))

g = ig.Graph(directed=False)
g.add_vertices(nv)

for i in range(len(g_edges)):
    g.add_edge(g_edges["from"][i], g_edges["to"][i])

g.es['weight'] = g_edges["w"]

In [ ]:
## read lat/lon position of nodes (intersections)
meta = pd.read_csv(datadir+'Reno/nodeloc.csv')

g.vs['longitude'] = list(meta['lon'])
g.vs['latitude'] = list(meta['lat'])
g.vs['layout'] = [(v['longitude'],v['latitude']) for v in g.vs]

In [ ]:
## keep giant connected component only
g = g.clusters().giant()

## Using betweenness without edge weights (road length)

We distinguish 4 types of nodes w.r.t. betweenness:
* the heaviest one (green)
* the very heavy ones (red), 99th percentile
* heavy ones (violet), 90th percentile
* others (black)

In [ ]:
## compute betweenness and plot distribution
bet = g.betweenness()
plt.hist(bet, 50);

In [ ]:
## color and size w.r.t. 4 types of nodes
top_node = max(bet)
very_heavy_usage = np.quantile(bet, 0.99)
heavy_usage = np.quantile(bet, 0.9)

g.vs['color'] = ["black" if b < heavy_usage else "violet" if b < very_heavy_usage else "green" if b == top_node else "red" for b in bet]
g.vs['size'] = [3 if b < heavy_usage else 5 if b < very_heavy_usage else 15 if b == top_node else 8 for b in bet]

In [ ]:
## plot highlighting intersections with high betweenness
ly = ig.Layout(g.vs['layout'])
ly.mirror(1)
ig.plot(g, layout=ly, vertex_size=g.vs['size'], vertex_color=g.vs['color'], edge_arrow_size=0.7, edge_arrow_width=0.7)

#### When we do map overlay we note that there is a river in the middle of the plot going from west to east.

In [ ]:
MAP_BOUNDS = ((39.5001-0.001, -119.802-0.001), (39.5435+0.001, -119.7065+0.001))
m_plot = flm.Map()

for v in g.vs:
    flm.Circle(
        (v['latitude'], v['longitude']),
        radius=10, color=v['color'], weight= v['size'],
        fill=True, fill_color=v['color']).add_to(m_plot)

for e in g.es:
    v1 = g.vs[e.source]
    v2 = g.vs[e.target]
    flm.PolyLine(
        [(v1['latitude'], v1['longitude']), (v2['latitude'], v2['longitude'])],
        color="black", weight=1).add_to(m_plot)

flm.Rectangle(MAP_BOUNDS, color="blue",weight=4).add_to(m_plot)
m_plot.fit_bounds(MAP_BOUNDS)
m_plot

## Using betweenness with edge weights (road length)


In [ ]:
## compute betweenness and plot distribution
bet = g.betweenness(weights=g.es['weight'])
plt.hist(bet, 50);

In [ ]:
## color w.r.t. 4 node types
top_node = max(bet)
very_heavy_usage = np.quantile(bet, 0.99)
heavy_usage = np.quantile(bet, 0.9)

g.vs['color'] = ["black" if b < heavy_usage else "violet" if b < very_heavy_usage else "green" if b == top_node else "red" for b in bet]
g.vs['size'] = [3 if b < heavy_usage else 5 if b < very_heavy_usage else 15 if b == top_node else 8 for b in bet]

In [ ]:
## plot highlighting intersections with high betweenness
ly = ig.Layout(g.vs['layout'])
ly.mirror(1)
ig.plot(g, layout=ly, vertex_size=g.vs['size'], vertex_color=g.vs['color'], edge_arrow_size=0.7, edge_arrow_width=0.7)

In [ ]:
MAP_BOUNDS = ((39.5001-0.001, -119.802-0.001), (39.5435+0.001, -119.7065+0.001))
m_plot = flm.Map()

for v in g.vs:
    flm.Circle(
        (v['latitude'], v['longitude']),
        radius=10, color=v['color'], weight= v['size'],
        fill=True, fill_color=v['color']).add_to(m_plot)

for e in g.es:
    v1 = g.vs[e.source]
    v2 = g.vs[e.target]
    flm.PolyLine(
        [(v1['latitude'], v1['longitude']), (v2['latitude'], v2['longitude'])],
        color="black", weight=1).add_to(m_plot)

flm.Rectangle(MAP_BOUNDS, color="blue",weight=4).add_to(m_plot)
m_plot.fit_bounds(MAP_BOUNDS)
m_plot